In [22]:
from __future__ import print_function 
import numpy as np
import tensorflow as tf 

import os
import sys

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Lambda, concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import SimpleRNN, GRU, LSTM, Bidirectional

import keras.backend as K

## Load the dataset

In [13]:
f = np.load('data_and_embedding.npz')

In [15]:
num_words = int(f['num_words'])
embedding_dim = int(f['embedding_dim'])
max_sequence_length = int(f['max_sequence_length'])

data = f['data']
labels = f['labels']

embedding_matrix = f['embedding_matrix']

In [35]:
validation_split = 0.2 
epochs = 60

In [18]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(validation_split * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

## Conv + RNN

### Build the model 

In [19]:
embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False)

In [20]:
def embedding_mean(x):
    return tf.reduce_mean(x, axis=1)

def conc_conv_lstm(x, y):
    return tf.concat([x, y], axis=0)

In [30]:
sequence_input = Input(shape=(max_sequence_length, ), dtype='int32')
# print(sequence_input.shape)
embedded_sequences = embedding_layer(sequence_input)
print(embedded_sequences.shape)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
print(x.shape)
x = Flatten()(x)
print(x.shape)
x = Dense(128, activation='relu')(x)

y = LSTM(50, dropout=0.2, recurrent_dropout=0.2)(embedded_sequences)
# y = Lambda(embedding_mean)(y)
print(y.shape)
z = concatenate([x, y])
print(z.shape)
preds = Dense(6, activation='softmax')(z)

model_conv_rnn = Model(sequence_input, preds)

(?, 1000, 100)
(?, 1, 128)
(?, ?)
(?, 50)
(?, 178)


In [31]:
model_conv_rnn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_10 (InputLayer)            (None, 1000)          0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 1000, 100)     2000000                                      
____________________________________________________________________________________________________
conv1d_26 (Conv1D)               (None, 996, 128)      64128                                        
____________________________________________________________________________________________________
max_pooling1d_25 (MaxPooling1D)  (None, 199, 128)      0                                            
___________________________________________________________________________________________

### Train the model

In [32]:
model_conv_rnn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
import time 
start_time = time.time()

model_conv_rnn.fit(x_train, y_train,
          batch_size=128,
          epochs=epochs,
          validation_data=(x_val, y_val))

print("Training time: ", time.time() - start_time)

Train on 311902 samples, validate on 77975 samples
Epoch 1/60
  4864/311902 [..............................] - ETA: 4600s - loss: 1.1952 - acc: 0.6338

### Save the model

In [ ]:
model_conv_rnn.save('models/ConvRNN.h5')